#  Profundización Análisis Stable Coins

En este notebook profundizaremos nuestro análisis sobre nuestras Stable Coins considerando algunos de los indicadores más importantes a las hora de invertir.

Indicadores:
- Market Capital 'PI_II_DA.ipynb'
- ATR Average True Range
- RSI Relative Strength Index

In [56]:
from pycoingecko import CoinGeckoAPI
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import time as t
import mplfinance as fplt

Creación de una instancia de CoinGeckoAPI para acceder a la API CoinGecko

In [57]:
cg = CoinGeckoAPI()

Comprobación de conexión con la API CoinGecko

In [58]:
cg.ping()

{'gecko_says': '(V3) To the Moon!'}

In [39]:
coin_categories = pd.DataFrame(cg.get_coins_categories_list())
coin_categories.head()

,category_id,name
0,aave-tokens,Aave Tokens
1,algorand-ecosystem,Algorand Ecosystem
2,alleged-sec-securities,Alleged SEC Securities
3,analytics,Analytics
4,animal-racing,Animal Racing


In [40]:
coin_categories[coin_categories['category_id'].str.contains('stable')]

,category_id,name
35,cny-stablecoin,CNY Stablecoin
64,eur-stablecoin,EUR Stablecoin
75,gbp-stablecoin,GBP Stablecoin
85,idr-stablecoin,IDR Stablecoin
97,jpy-stablecoin,JPY Stablecoin
101,krw-stablecoin,KRW Stablecoin
172,stablecoins,Stablecoins
190,try-stablecoins,TRY Stablecoin
191,usd-stablecoin,USD Stablecoin


In [41]:
stable_market_cap_data = pd.DataFrame(cg.get_coins_markets(vs_currency = 'usd', category = 'stablecoins')).sort_values('market_cap', ascending=False)
stable_df = stable_market_cap_data[['name', 'id', 'current_price', 'market_cap', 'circulating_supply']].head(10)
display(stable_df)

,name,id,current_price,market_cap,circulating_supply
0,Tether,tether,0.998542,8.306072e+10,8.318200e+10
1,USD Coin,usd-coin,0.999885,2.603399e+10,2.603333e+10
2,Dai,dai,0.999268,3.926286e+09,3.929161e+09
3,Binance USD,binance-usd,0.999354,3.335357e+09,3.337600e+09
4,TrueUSD,true-usd,0.998589,2.748711e+09,2.751975e+09
5,Frax,frax,0.996993,8.103257e+08,8.126414e+08
6,USDD,usdd,0.996535,7.228458e+08,7.253320e+08
7,Pax Dollar,paxos-standard,0.999132,5.058243e+08,5.061000e+08
8,PAX Gold,pax-gold,1898.050000,4.715887e+08,2.483434e+05
9,Tether Gold,tether-gold,1903.430000,4.691881e+08,2.465243e+05


Ahora organicemos las monedas en forma de lista

In [42]:
stable_list = stable_df.id.tolist()
print(stable_list)

['tether', 'usd-coin', 'dai', 'binance-usd', 'true-usd', 'frax', 'usdd', 'paxos-standard', 'pax-gold', 'tether-gold']


A continuación crearemos un DataFrame vacío con columnas para almacenar información sobre los precios de nuestra selección de criptomonedas a lo largo del tiempo. 
Posteriormente, este DataFrame se llenará con los datos específicos de cada criptomoneda determinada.

In [59]:
data = {'Date':[], 'Open':[], 'High':[], 'Low':[], 'Close':[], 'id':[]}
df = pd.DataFrame.from_dict(data)
df

,Date,Open,High,Low,Close,id


El siguiente fragmento de código iterará a través de 'lista_monedas' y para cada una de las moneda seleccionadas, estará obteniendo datos históricos de precios, calculando el indicador ATR (Average True Range) y el indicador RSI (Relative Strength Index), y luego agregará estos datos al DataFrame df.

In [60]:
# Iteración a través de las monedas:
lista_monedas = ['tether', 'usd-coin', 'dai', 'binance-usd', 'true-usd', 'frax', 'usdd', 'paxos-standard', 'pax-gold', 'tether-gold']
for moneda in lista_monedas:
    
# Obtención de datos históricos de precios (OHLC: Open, High, Low, Close):
    ohlc_data = cg.get_coin_ohlc_by_id(id = moneda, vs_currency = 'usd', days = '180')
    ohlc_data_frame = pd.DataFrame(data = ohlc_data, columns = ['Date', 'Open', 'High' ,'Low', 'Close'])
    ohlc_data_frame['Date'] = ohlc_data_frame['Date'].apply(lambda x: dt.datetime.fromtimestamp(x/1000).strftime('%m-%d-%Y %H:%M:%S'))
    ohlc_data_frame['Date'] = pd.to_datetime(ohlc_data_frame['Date'])
    ohlc_data_frame = ohlc_data_frame.set_index('Date')
    ohlc_data_frame['id'] = moneda

# Cálculo del indicador ATR (Average True Range):
    def wwma(values, n):
        return values.ewm(alpha=1/n, adjust=False).mean()
    def atr(df, n=14):
        data = df.copy()
        high = data['High']
        low = data['Low']
        close = data['Close']
        data['tr0'] = abs(high - low)
        data['tr1'] = abs(high - close.shift())
        data['tr2'] = abs(low - close.shift())
        tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
        atr = wwma(tr, n)
        return atr
    ohlc_data_frame['ATR'] = atr(ohlc_data_frame)
    ohlc_data_frame['change'] = ohlc_data_frame['Close'].diff()
    ohlc_data_frame['gain'] = ohlc_data_frame.change.mask(ohlc_data_frame.change < 0, 0.0)
    ohlc_data_frame['loss'] = -ohlc_data_frame.change.mask(ohlc_data_frame.change > 0, -0.0)

# Cálculo del indicador RSI (Relative Strength Index):
    def rma(x, n):
        """Running moving average"""
        a = np.full_like(x, np.nan)
        a[n] = x[1:n+1].mean()
        for i in range(n+1, len(x)):
            a[i] = (a[i-1] * (n - 1) + x[i]) / n
        return a
    ohlc_data_frame['avg_gain'] = rma(ohlc_data_frame.gain.to_numpy(), 14)
    ohlc_data_frame['avg_loss'] = rma(ohlc_data_frame.loss.to_numpy(), 14)
    ohlc_data_frame['rs'] = ohlc_data_frame.avg_gain / ohlc_data_frame.avg_loss
    ohlc_data_frame['RSI'] = 100 - (100 / (1 + ohlc_data_frame.rs))
    ohlc_data_frame = ohlc_data_frame.drop(['change', 'gain', 'loss', 'avg_gain', 'avg_loss', 'rs'], axis=1)
    df = pd.concat([df, ohlc_data_frame], ignore_index=True)

Repasemos lo realizado:

1. Iteración a través de las monedas: para cada moneda en la lista, se realizarán una serie de operaciones.

2. Obtención de datos históricos de precios: Aquí obtenemos los datos históricos de precios (OHLC: Open, High, Low, Close) de la moneda actual usando la API CoinGecko. Luego, estos datos se convierten en un DataFrame llamado 'ohlc_data_frame'. La columna Date se ajusta para convertir los valores de tiempo en fechas legibles, y la columna id se establece en el identificador de la moneda actual.

3. Cálculo del indicador ATR (Average True Range): Aquí definimos funciones para calcular el indicador ATR. Luego, calculamos los valores ATR para el DataFrame 'ohlc_data_frame' y se agregan como una columna llamada 'ATR'.

4. Cálculo del indicador RSI (Relative Strength Index): Aquí calculamos el indicador RSI para el DataFrame 'ohlc_data_frame'. Calculamos las diferencias entre los precios de cierre, los valores de ganancias y pérdidas, y calculamos el RSI usando estos valores. Luego, se eliminan las columnas temporales utilizadas para calcular el RSI, y el DataFrame 'ohlc_data_frame' ahora contiene los precios históricos, el indicador ATR y el indicador RSI para la moneda actual.

 Por último, a continuación, exportamos nuestro DataFrame que contendrá los datos históricos de precios, así como los indicadores ATR y RSI calculados para cada moneda en la lista 'lista_monedas'.

In [54]:
df.to_csv('olhc_monedas.csv', index=False)